In [102]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import copy

In [103]:
from pyomo.environ import *
from pyomo.environ import RangeSet
from pyomo.environ import value

In [104]:
A_url = "https://raw.githubusercontent.com/FarshidNazemi/Plastic-Packaging/main/csv-files/Technology%20Matrix%20(A)%20-%20Design.csv"
B_url = "https://raw.githubusercontent.com/FarshidNazemi/Plastic-Packaging/main/csv-files/Environmental%20Matrix%20(B)%20-%20Design.csv"
C_url = "https://raw.githubusercontent.com/FarshidNazemi/Plastic-Packaging/main/csv-files/Characterization%20Matrix%20(C)%20-%20Design.csv"

In [105]:
A_df = pd.read_csv(A_url,header=None)
B_df = pd.read_csv(B_url,header=None)
C_df = pd.read_csv(C_url,header=None)

In [106]:
A_df_org = A_df
B_df_org = B_df
C_df_org = C_df

In [107]:
#Building A matrix
# Step 1: Delete the first 4 columns
A_df = A_df.drop(A_df.columns[:4], axis=1)

# Step 2: Delete the first 4 rows (first row is heading, so put 3)
A_df = A_df.iloc[4:]

#Replacing empty values with zero and getting the final A matrix
A=A_df
A=A.replace(np.nan, 0)
A=np.array(A,dtype='float64')

In [108]:
#Building B matrix
# Step 1: Delete the first 6 columns
B_df = B_df.drop(B_df.columns[:6], axis=1)

# Step 2: Delete the first 4 rows (first row is heading, so put 3)
B_df = B_df.iloc[4:]

#Replacing empty values with zero and getting the final B matrix
B=B_df
B=B.replace(np.nan, 0)
B=np.array(B,dtype='float64')

In [109]:
#Building C matrix
# Step 1: Delete the first 6 columns
C_df = C_df.drop(C_df.columns[:6], axis=1)

# Step 2: Delete the first 5 rows (first row is heading, so put 3)
C_df = C_df.iloc[5:]

#Replacing empty values with zero and getting the final B matrix
C=C_df
C=C.replace(np.nan, 0)
C=np.array(C,dtype='float64')

In [130]:
#Functional Unit
# F = 168.450 million metric tons = 168,450,000 metric tons = 168,450,000,000 kg
F=168450000000
F=int(F)
#defining f matrix
f=np.zeros(len(A))
f[0]=F

In [131]:
C_gwp= np.transpose(C)[[0]]
coef_GWP=C_gwp@B
coef_GWP=np.array(coef_GWP)
coef_GWP = coef_GWP.reshape(-1)

In [126]:
#Model Formulation
# Create the model
model = ConcreteModel()
# List of processes with negative scaling factor due to open loop recovery and substitution approach
negative_s_indices = []
positive_s_indices = []
all_s_indices = []
search_elements_zero_or_negative = [
    'textile production, nonwoven polyester, needle-punched | textile, nonwoven polyester | APOS, S',
    'market for sawlog and veneer log, softwood, debarked, measured as solid wood | sawlog and veneer log, softwood, debarked, measured as solid wood | APOS, S',
    'pitch production, petroleum refinery operation | pitch | APOS, S',
    'lignite mine operation | lignite | APOS, S',
    'naphtha production, petroleum refinery operation | naphtha | APOS, S',
    'sodium sulfate production, from natural sources | sodium sulfate, anhydrite | APOS, S',
    'methanol production | methanol | APOS, S',
    'petroleum production, onshore | petroleum | APOS, S'
]

search_elements_zero_or_negative_or_positive = [
    'textile production, nonwoven polyester, needle-punched | textile, nonwoven polyester | APOS, S',
    'market for sawlog and veneer log, softwood, debarked, measured as solid wood | sawlog and veneer log, softwood, debarked, measured as solid wood | APOS, S',
    'pitch production, petroleum refinery operation | pitch | APOS, S',
    'lignite mine operation | lignite | APOS, S',
    'market group for electricity, medium voltage | electricity, medium voltage | APOS, S',
    'heat production, natural gas, at industrial furnace low-NOx >100kW | heat, district or industrial, natural gas | APOS, S',
    'naphtha production, petroleum refinery operation | naphtha | APOS, S',
    'sodium sulfate production, from natural sources | sodium sulfate, anhydrite | APOS, S',
    'methanol production | methanol | APOS, S',
    'petroleum production, onshore | petroleum | APOS, S'
]

# Search for elements in the first row of the DataFrame
negative_s_indices = [A_df_org.iloc[0,:].tolist().index(elem) for elem in search_elements_zero_or_negative]
negative_s_indices = [i-3 for i in negative_s_indices]
negative_or_positive_s_indices = [A_df_org.iloc[0,:].tolist().index(elem) for elem in search_elements_zero_or_negative_or_positive]
negative_or_positive_s_indices = [i-3 for i in negative_or_positive_s_indices]
all_s_indices = list(range(1, len(np.transpose(A))+1))
positive_s_indices = [index for index in all_s_indices if index not in negative_or_positive_s_indices]

# Define the decision variable
model.set_s = RangeSet(len(np.transpose(A)))
model.s = Var(model.set_s)
model.set_negative_scale = Set(initialize=negative_s_indices)
model.set_positive_scale = Set(initialize=positive_s_indices)

#model constraint: As = f and s>=0
model.set_balance = RangeSet(len(f))
def balance(model, p): # As = f
    return sum(A[p-1,i-1]*model.s[i] for i in model.set_s) == f[p-1]
def negative_scale(model, i):
    return (model.s[i]<=0)
def positive_scale(model, i):
    return (model.s[i]>=0)

model.balance_constraints = Constraint(model.set_balance, rule=balance)
model.negative_scale_constraints = Constraint(model.set_negative_scale, rule=negative_scale)
model.positive_scale_constraints = Constraint(model.set_positive_scale, rule=positive_scale)

model.obj = Objective(expr = sum(coef_GWP[i-1]*model.s[i] for i in model.set_s), sense=minimize)
#Solver
solver = SolverFactory('glpk')
solver.solve(model) # solves and updates instance

{'Problem': [{'Name': 'unknown', 'Lower bound': 2.48559955244727, 'Upper bound': 2.48559955244727, 'Number of objectives': 1, 'Number of constraints': 259, 'Number of variables': 144, 'Number of nonzeros': 610, 'Sense': 'minimize'}], 'Solver': [{'Status': 'ok', 'Termination condition': 'optimal', 'Statistics': {'Branch and bound': {'Number of bounded subproblems': 0, 'Number of created subproblems': 0}}, 'Error rc': 0, 'Time': 0.03033614158630371}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}

In [113]:
#total CO2 emission (kg CO2eq)
value(model.obj)

418699244609.7435

In [114]:
#kg CO2 eq per kg of packaging product
value(model.obj)/168450000000

2.485599552447275

In [115]:
#share of global CO2 emissions
value(model.obj)/55000000000000

0.007612713538358973

In [116]:
scaling_factors = []
for j in model.s:
    scaling_factors.append(model.s[j].value)  

In [117]:
#1. monomer production
#ethylene
ethylene_s_indices = []
scaling_factors_ethylene = []
search_elements_ethylene = [
    'Bio-ethylene production (monomer)',
    'Fossil-ethylene production',
    'pyrolysis (monomer production), HDPE sorted',
    'pyrolysis (monomer production), LDPE sorted'
]
ethylene_s_indices = [A_df_org.iloc[0,:].tolist().index(elem) for elem in search_elements_ethylene]
ethylene_s_indices = [i-3 for i in ethylene_s_indices]
for j in ethylene_s_indices:
    scaling_factors_ethylene.append(model.s[j].value)
scaling_factors_ethylene

[0.0, 74362954650.7235, 0.0, 0.0]

In [118]:
#propylene
propylene_s_indices = []
scaling_factors_propylene = []
search_elements_propylene = [
    'Bio-propylene production (monomer)',
    'Fossil-propylene production',
    'pyrolysis (monomer production), PP sorted',
]
propylene_s_indices = [A_df_org.iloc[0,:].tolist().index(elem) for elem in search_elements_propylene]
propylene_s_indices = [i-3 for i in propylene_s_indices]
for j in propylene_s_indices:
    scaling_factors_propylene.append(model.s[j].value)
scaling_factors_propylene   

[0.0, 48857376126.4011, 0.0]

In [119]:
#MEG
MEG_s_indices = []
scaling_factors_MEG = []
search_elements_MEG = [
    'Bio-MEG production (monomer)',
    'Fossil-MEG production',
    'depolymerization, enzymatic hydrolysis, PET sorted',
    'depolymerization, glycolysis, PET sorted'
]
MEG_s_indices = [A_df_org.iloc[0,:].tolist().index(elem) for elem in search_elements_MEG]
MEG_s_indices = [i-3 for i in MEG_s_indices]
for j in MEG_s_indices:
    scaling_factors_MEG.append(model.s[j].value)
scaling_factors_MEG   

[0.0, 12986265878.4695, 0.0, 0.0]

In [120]:
#styrene
styrene_s_indices = []
scaling_factors_styrene = []
search_elements_styrene = [
    'Bio-styrene production',
    'Fossil-styrene production',
    'chemical recycling with metal oxide, PS sorted',
]
styrene_s_indices = [A_df_org.iloc[0,:].tolist().index(elem) for elem in search_elements_styrene]
styrene_s_indices = [i-3 for i in styrene_s_indices]
for j in styrene_s_indices:
    scaling_factors_styrene.append(model.s[j].value)
scaling_factors_styrene 

[0.0, 8490211206.10363, 0.0]

In [121]:
#vinyl chloride
vinyl_chloride_s_indices = []
scaling_factors_vinyl_chloride = []
search_elements_vinyl_chloride = [
    'Bio-vinyl chloride production',
    'Fossil-vinyl chloride production',
]
vinyl_chloride_s_indices = [A_df_org.iloc[0,:].tolist().index(elem) for elem in search_elements_vinyl_chloride]
vinyl_chloride_s_indices = [i-3 for i in vinyl_chloride_s_indices]
for j in vinyl_chloride_s_indices:
    scaling_factors_vinyl_chloride.append(model.s[j].value)
scaling_factors_vinyl_chloride  

[0.0, 3564093390.09828]

In [122]:
# 2. polymers
#polyethylene
scaling_factors_polyethylene = []
polyethylene_s_indices = []
search_elements_polyethylene = [
    'advanced recycling, delamination, multi-layer plastic packaging',
    'advanced recycling, STRAP, multi-layer plastic packaging',
    'LDPE production',
    'mechanical recycling, LDPE sorted'
]
polyethylene_s_indices = [A_df_org.iloc[0,:].tolist().index(elem) for elem in search_elements_polyethylene]
polyethylene_s_indices = [i-3 for i in polyethylene_s_indices]
for j in polyethylene_s_indices:
    scaling_factors_polyethylene.append(model.s[j].value)
scaling_factors_polyethylene

[0.0, 0.0, 34568875648.3937, 0.0]

In [123]:
#polypropylene
scaling_factors_polypropylene = []
polypropylene_s_indices = []
search_elements_polypropylene = [
    'advanced recycling, delamination, multi-layer plastic packaging',
    'advanced recycling, STRAP, multi-layer plastic packaging',
    'PP production',
    'mechanical recycling, PP sorted'
]
polypropylene_s_indices = [A_df_org.iloc[0,:].tolist().index(elem) for elem in search_elements_polypropylene]
polypropylene_s_indices = [i-3 for i in polypropylene_s_indices]
for j in polypropylene_s_indices:
    scaling_factors_polypropylene.append(model.s[j].value)
scaling_factors_polypropylene

[0.0, 0.0, 49487851352.6337, 0.0]

In [73]:
#3. mixed plastic collected waste (sorting, energy recovery, landfill, mismanaged)

scaling_factors_mixed = []
mixed_s_indices = []
search_elements_mixed = [
    'incineration, electricity recovery, plastic packaging waste',
    'landfill, plastic packaging waste',
    'Mismanaged waste treatment, uncontrolled landfill, plastic packaging waste',
    'sorting, non-optical manual and automatic sorter (commercial)',
    'sorting, optiacl sorter (commercial)',
    'sorting, optical with film sorter (commercial)',
    'sorting, optical with film, multilayer, and AI sorter (emerging)'
]
mixed_s_indices = [A_df_org.iloc[0,:].tolist().index(elem) for elem in search_elements_mixed]
mixed_s_indices = [i-3 for i in mixed_s_indices]
for j in mixed_s_indices:
    scaling_factors_mixed.append(model.s[j].value)
scaling_factors_mixed

[0.0, 160027500000.0, 0.0, 0.0, 0.0, 0.0, 0.0]

In [74]:
# Sankey Diagram
#data
label = ["Resources","Feedstocks","Fossil-based monomers","Bio-based monomers","Monomers",
        "Polymers","Products","Use & collection","Sorting","Advanced Recycling", "Mechanical Recycling",
        "Chemical Recycling", "Downcycling", "Energy Recovery", "Landfill", "Mismanaged Waste",
        "Macroplastics", "Microplastics"]

#1: Resources
#2: Feedstocks
#3: Fossil-based monomers
#4: Bio-based monomers
#5: Monomers
#6: Polymers
#7: Products
#8: Use & collection
#9: Sorting
#10: Advanced Recycling
#11: Mechanical Recycling
#12: Chemical Recycling
#13: Downcycling
#14: Energy Recovery
#15: Landfill
#16: Mismanaged Waste
#17: Macroplastics
#18: Microplastics

source = [1,2,2,3,4,5,6,7,8, 8, 8, 8, 8, 9, 9, 9, 9, 9, 9, 9,10,10,10,10,11,11,11,11,12,12,12,12,12,13,13,13,16,17]
target = [2,3,4,5,5,6,7,8,9,14,15,16,17,10,11,12,13,14,15,16, 7,14,15,16, 7,14,15,16, 5, 2,14,15,16,14,15,16,17,18]
value = [
    round(abs(A[A_df_org.index[A_df_org.iloc[:, 0] == 'market for maleic anhydride | maleic anhydride | APOS, S'].tolist()[0] - 10,
                   A_df_org.index[A_df_org.iloc[:, 0] == 'market for maleic anhydride | maleic anhydride | APOS, S'].tolist()[0] - 10])
                      * abs(s[A_df_org.index[A_df_org.iloc[:, 0] == 'market for maleic anhydride | maleic anhydride | APOS, S'].tolist()[0] - 10])/1000
                      + abs(A_modified[A_df_org.index[A_df_org.iloc[:, 0] == 'market for nylon 6 | nylon 6 | APOS, S'].tolist()[0] - 10,
                                        A_df_org.index[A_df_org.iloc[:, 0] == 'market for nylon 6 | nylon 6 | APOS, S'].tolist()[0] - 10])
                      * abs(s[A_df_org.index[A_df_org.iloc[:, 0] == 'market for nylon 6 | nylon 6 | APOS, S'].tolist()[0] - 10])/1000


SyntaxError: unexpected EOF while parsing (2032102939.py, line 35)